In [ ]:
import scipy.stats as stats

def ab_test_proportions(control_visitors, control_converts, 
                        variant_visitors, variant_converts,
                        confidence_level=0.95):
    """
    Two-proportion z-test for conversion rates
    """
    # Conversion rates
    p1 = control_converts / control_visitors
    p2 = variant_converts / variant_visitors
    
    # Pooled probability
    p_pool = (control_converts + variant_converts) / (control_visitors + variant_visitors)
    
    # Standard error
    se = np.sqrt(p_pool * (1 - p_pool) * (1/control_visitors + 1/variant_visitors))
    
    # Z-score
    z = (p2 - p1) / se
    
    # P-value (two-tailed)
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    # Confidence interval
    z_crit = stats.norm.ppf(1 - (1 - confidence_level) / 2)
    ci_low = (p2 - p1) - z_crit * se
    ci_high = (p2 - p1) + z_crit * se
    
    # Relative lift
    lift = (p2 - p1) / p1 * 100
    
    results = {
        'control_rate': p1,
        'variant_rate': p2,
        'relative_lift': lift,
        'p_value': p_value,
        'significant': p_value < (1 - confidence_level),
        'confidence_interval': (ci_low, ci_high),
        'z_score': z
    }
    
    return results

# Example usage
result = ab_test_proportions(
    control_visitors=10000, control_converts=500,
    variant_visitors=10000, variant_converts=550
)

print(f"Control: {result['control_rate']:.1%}")
print(f"Variant: {result['variant_rate']:.1%}")
print(f"Lift: {result['relative_lift']:.1f}%")
print(f"P-value: {result['p_value']:.4f}")
print(f"Significant: {result['significant']}")

# Bayesian approach for small samples
from scipy.stats import beta

def bayesian_ab_test(control_converts, control_visitors,
                     variant_converts, variant_visitors,
                     num_simulations=100000):
    
    # Beta distributions for each variant
    control_dist = beta(control_converts + 1, control_visitors - control_converts + 1)
    variant_dist = beta(variant_converts + 1, variant_visitors - variant_converts + 1)
    
    # Monte Carlo simulation
    control_samples = control_dist.rvs(num_simulations)
    variant_samples = variant_dist.rvs(num_simulations)
    
    # Probability variant is better
    prob_variant_better = (variant_samples > control_samples).mean()
    
    # Expected lift
    lift_samples = (variant_samples - control_samples) / control_samples
    expected_lift = np.mean(lift_samples) * 100
    lift_ci = np.percentile(lift_samples * 100, [2.5, 97.5])
    
    return {
        'prob_variant_better': prob_variant_better,
        'expected_lift': expected_lift,
        'lift_95_ci': lift_ci
    }